In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import QuantileTransformer
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import os

df = pd.read_csv("../data/raw/insurance.csv")


ImportError: DLL load failed while importing _cyutility: An Application Control policy has blocked this file.

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
print("=== Missing Values ===")
print(df.isnull().sum())
print("\n")

In [ ]:
#Encode categorical variables
# 'sex', 'smoker', 'region' are categorical
df_encoded = df.copy()

In [ ]:
# Convert 'sex' and 'smoker' to binary
df_encoded['sex'] = df_encoded['sex'].map({'male': 1, 'female': 0})
df_encoded['smoker'] = df_encoded['smoker'].map({'yes': 1, 'no': 0})


In [ ]:
# One-hot encode 'region'
df_encoded = pd.get_dummies(df_encoded, columns=['region'], drop_first=True)


In [ ]:
print("=== Encoded Data Sample ===")
print(df_encoded.head())
print("\n")

In [ ]:
#Inspect outliers for 'charges'
plt.figure(figsize=(8,5))
sns.boxplot(x=df_encoded['charges'])
plt.title("Boxplot of Insurance Charges (Outlier Detection)")
plt.show()

In [ ]:
#basic statistics to identify extreme values
print("=== Charges Statistics ===")
print(df_encoded['charges'].describe())

In [ ]:
#Distribution of charges
plt.figure(figsize=(8,5))
sns.histplot(df_encoded['charges'], bins=50, kde=True)
plt.title("Distribution of Insurance Charges")
plt.xlabel("Charges (USD)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Check skewness
print("Skewness of Charges:", df_encoded['charges'].skew())
print("Kurtosis of Charges:", df_encoded['charges'].kurtosis())
print("\n")

In [ ]:
# Initialize transformer
qt = QuantileTransformer(
    output_distribution='normal',
    random_state=0
)

# Apply quantile transformation to charges
df['charges_qt'] = qt.fit_transform(
    df[['charges']]
).flatten()

# Check skewness
print(f"Skewness after Quantile Transformation: {df['charges_qt'].skew():.5f}")

In [ ]:
# Histogram of quantile-transformed charges
plt.figure()
plt.hist(df['charges_qt'], bins=30)
plt.title('Histogram of Quantile-Transformed Charges')
plt.xlabel('charges (quantile-transformed)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#Boxplots by smoker
plt.figure(figsize=(8,5))
sns.boxplot(x='smoker', y='charges', data=df_encoded)
plt.title("Insurance Charges by Smoking Status")
plt.xlabel("Smoker (1=Yes, 0=No)")
plt.ylabel("Charges (USD)")
plt.show()

In [ ]:
#Boxplots by region (one-hot columns)
region_cols = [col for col in df_encoded.columns if 'region_' in col]

In [ ]:
#Melt dataframe for seaborn boxplot
df_melt = df_encoded.melt(id_vars='charges', value_vars=region_cols, var_name='region', value_name='present')
df_melt = df_melt[df_melt['present']==1]  # keep only rows where region is present

plt.figure(figsize=(10,6))
sns.boxplot(x='region', y='charges', data=df_melt)
plt.title("Insurance Charges by Region")
plt.xlabel("Region")
plt.ylabel("Charges (USD)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Correlation matrix
plt.figure(figsize=(10,8))
corr = df_encoded.corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title("Correlation Matrix of Variables")
plt.show()

In [ ]:
#t-test: Smoker vs Non-Smoker
smoker_charges = df_encoded[df_encoded['smoker']==1]['charges']
nonsmoker_charges = df_encoded[df_encoded['smoker']==0]['charges']

t_stat, p_val = stats.ttest_ind(smoker_charges, nonsmoker_charges, equal_var=False)
print("=== T-Test: Charges by Smoker Status ===")
print(f"T-statistic: {t_stat:.3f}")
print(f"P-value: {p_val:.3f}")
if p_val < 0.05:
    print("Result: Significant difference in charges between smokers and non-smokers.\n")
else:
    print("Result: No significant difference in charges.\n")

In [ ]:
#ANOVA: Region Differences
# Use the original 'region' column from df (categorical, not one-hot)
anova_model = ols('charges ~ C(region)', data=df).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)
print("=== ANOVA: Charges by Region ===")
print(anova_table)
print("\n")

In [ ]:
# Tukey HSD post-hoc test for regional differences in charges
tukey = pairwise_tukeyhsd(
    endog=df['charges'],      # dependent variable
    groups=df['region'],      # grouping variable
    alpha=0.05
)

print(tukey)


In [ ]:
y = df['charges']

X = df[['age', 'bmi', 'children', 'sex', 'smoker', 'region']]


In [ ]:
X = pd.get_dummies(
    X,
    columns=['sex', 'smoker', 'region'],
    drop_first=True
)


In [ ]:
X = X.astype(float)


In [ ]:
X = sm.add_constant(X)


In [ ]:
model = sm.OLS(y, X).fit()
print(model.summary())


In [ ]:
# Initialize QuantileTransformer to normalize the distribution
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)

# Fit and transform 'charges', store as new column
df['charges_quant'] = quantile_transformer.fit_transform(df['charges'].values.reshape(-1, 1)).flatten()

# Check skewness
print("Skewness after Quantile Transformation:", df['charges_quant'].skew())


In [ ]:
# One-hot encoding, drop first to avoid multicollinearity
categorical_cols = ['sex', 'smoker', 'region']

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)



In [ ]:
# Exclude original 'charges' column
X = df_encoded.drop(columns=['charges', 'charges_quant'])
y = df_encoded['charges_quant']

# Convert all columns to float (safe for OLS)
X = X.astype(float)
y = y.astype(float)



In [ ]:
X = sm.add_constant(X)



In [ ]:
model_transformed = sm.OLS(y, X).fit()



In [ ]:
print(model_transformed.summary())


In [ ]:
plt.hist(model_transformed.resid, bins=30)
plt.title("Residuals of Quantile-Transformed Charges Regression")
plt.show()


In [ ]:
print("Skewness of residuals:", pd.Series(model_transformed.resid).skew())


1. Dependent Variable

Original regression: charges (raw insurance costs)

Transformed regression: charges_quant (Quantile-transformed to approximate a normal distribution)

Purpose of transformation: To reduce skewness and improve the validity of regression assumptions (especially normality of residuals).

2. R-squared

Original: 0.751

Transformed: 0.764

Interpretation: The transformed model explains slightly more variance in the dependent variable (~76.4% vs 75.1%). Transformation didn’t reduce model fit; in fact, it slightly improved it.

3. Coefficients
Variable	      Original Coef	           Transformed Coef	          Interpretation

const	          -11,940	               -2.344	                  Intercept differs in scale due to transformation.

age	              256.86	               0.038	                  Positive effect; older people have higher charges.

bmi	              339.19	               0.018	                  Higher BMI → higher charges. Effect size smaller in                                                                       transformed scale.

children	      475.50	               0.104	                  More children → slightly higher charges.

sex_male	      -131.31	               -0.108	                  Male gender slightly reduces charges; now                                                                                 statistically significant in transformed regression.

smoker_yes	      23,850	               1.708	                  Huge positive effect of smoking persists;                                                                                 transformation preserves strong relationship.

region_nw	      -352.96	               -0.069	                  Slight negative effect; marginally non-significant                                                                        in transformed model.

region_se	      -1,035.02	               -0.198	                  Significant negative effect persists.

region_sw	      -960.05	               -0.148	                  Significant negative effect persists.

Observation:

Direction of effects is consistent between models.

Magnitudes are not comparable directly because of the transformation scale.

Statistical significance changed slightly: sex_male became significant, region_northwest borderline.

4. Residual Diagnostics

Skewness reduced in the dependent variable, but residual skew remains (Skew: 1.147 vs 1.211 before).

Kurtosis increased (10.26 vs 5.65), indicating heavier tails in residuals after transformation.

Omnibus and Jarque-Bera tests still reject normality, but transformation reduces extreme skew influence and stabilizes variance.

5. F-statistic

Original: 500.8

Transformed: 536.6

Interpretation: Model overall is highly significant in both cases. Transformation slightly increased F-statistic.

In [ ]:
df_encoded.dtypes


In [ ]:
# Ensure all dummies are numeric
categorical_cols = ['sex_male', 'smoker_yes', 
                    'region_northwest', 'region_southeast', 'region_southwest']

for col in categorical_cols:
    df_encoded[col] = pd.to_numeric(df_encoded[col], errors='coerce')


In [ ]:
df_encoded[categorical_cols].dtypes


In [ ]:
# Convert all boolean columns to int (0/1)
for col in categorical_cols:
    df_encoded[col] = df_encoded[col].astype(int)

# Confirm conversion
print(df_encoded[categorical_cols].dtypes)


In [ ]:
# Apply log transformation
df_encoded['charges_log'] = np.log(df_encoded['charges'])

# Define features and target
X = df_encoded[['age', 'bmi', 'children', 
                'sex_male', 'smoker_yes', 
                'region_northwest', 'region_southeast', 'region_southwest']]
X = sm.add_constant(X)
y_log = df_encoded['charges_log']

# Fit OLS regression on log-transformed charges
model_log = sm.OLS(y_log, X).fit()

# Show regression summary
print(model_log.summary())

In [ ]:
# Ensure categorical variables are numeric
for col in categorical_cols:
    df_encoded[col] = df_encoded[col].astype(int)

# Predictor matrix
X = df_encoded[['age', 'bmi', 'children', 
                'sex_male', 'smoker_yes', 
                'region_northwest', 'region_southeast', 'region_southwest']]
X = sm.add_constant(X)

# Dependent variable: log-transformed charges
y_log = np.log(df_encoded['charges'])

In [ ]:
# Robust regression
# --------------------------
robust_model = sm.RLM(y_log, X, M=sm.robust.norms.HuberT()).fit()

# Show summary
print(robust_model.summary())

In [ ]:
# Fit robust model again (if not already)
#X = df_encoded[['age', 'bmi', 'children', 'sex_male', 'smoker_yes', 
                #'region_northwest', 'region_southeast', 'region_southwest']]
#X = sm.add_constant(X)
#y = df_encoded['charges']

rlm_model = sm.RLM(y, X, M=sm.robust.norms.HuberT()).fit()

# Predicted values
y_pred = rlm_model.fittedvalues

# Residuals
residuals = y - y_pred

# Residual plot
plt.figure(figsize=(10,6))
sns.scatterplot(x=y_pred, y=residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residual Plot for Robust Linear Model")
plt.show()

# Histogram of residuals
plt.figure(figsize=(8,5))
sns.histplot(residuals, kde=True)
plt.title("Residual Distribution (RLM)")
plt.show()


In [ ]:
# Summary table
summary_table = rlm_model.summary2().tables[1]
print(summary_table)


In [ ]:
# Extract coefficients, standard errors, z-values, p-values, and CIs
rlm_summary = rlm_model.summary2().tables[1]

# Keep only relevant columns and rename for clarity
report_table = rlm_summary[['Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]']]
report_table.columns = ['Effect Size', 'Std Error', 'z-value', 'p-value', 'CI Lower', 'CI Upper']

# Round for readability
report_table = report_table.round(2)

# Display table
print(report_table)


In [ ]:
# Extract fitted values and residuals
fitted = rlm_model.fittedvalues
residuals = rlm_model.resid

# 1. Residuals vs Fitted
plt.figure(figsize=(8,5))
sns.scatterplot(x=fitted, y=residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

# 2. Histogram of Residuals
plt.figure(figsize=(8,5))
sns.histplot(residuals, bins=30, kde=True)
plt.xlabel('Residuals')
plt.title('Histogram of Residuals')
plt.show()

# 3. Q-Q Plot for Residuals (normality check)
import statsmodels.api as sm
sm.qqplot(residuals, line='45', fit=True)
plt.title('Q-Q Plot of Residuals')
plt.show()

# 4. Optional: Residuals vs Predictors
predictors = ['age','bmi','children','sex_male','smoker_yes',
              'region_northwest','region_southeast','region_southwest']

plt.figure(figsize=(12,8))
for i, col in enumerate(predictors):
    plt.subplot(3,3,i+1)
    sns.scatterplot(x=df_encoded[col], y=residuals, alpha=0.6)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel(col)
    plt.ylabel('Residuals')
plt.tight_layout()
plt.show()


In [ ]:
# Independent variable
X_age = df_encoded[['age']]
X_age = sm.add_constant(X_age)  # add intercept

# Dependent variable
y_charges = df_encoded['charges']


In [ ]:
# Robust linear regression using HuberT
rlm_age = sm.RLM(y_charges, X_age, M=sm.robust.norms.HuberT())
rlm_age_fit = rlm_age.fit()
print(rlm_age_fit.summary())


In [ ]:
sns.scatterplot(x='age', y='charges', data=df_encoded, alpha=0.5)
sns.lineplot(x=df_encoded['age'], y=rlm_age_fit.fittedvalues, color='red')
plt.title("Age vs Insurance Charges (Robust Regression Fit)")
plt.xlabel("Age")
plt.ylabel("Charges")
plt.show()


In [ ]:
feature_cols = [
    'age',
    'bmi',
    'children',
    'sex_male',
    'smoker_yes',
    'region_northwest',
    'region_southeast',
    'region_southwest'
]

X = df_encoded[feature_cols]
y = df_encoded['charges']   # raw charges only



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
scaler = # StandardScaler() / MinMaxScaler()
scaler.fit(X_train)
scaler.transformn(X_train) # np.array -> X_train_df
scaler.transform(X_test) # np.array    -> X_test_df

qantile = # QuantileTransformer()
quantile.fit(y_train)
quantile.transform(y_train) # np.array -> y_train_Series

In [ ]:
# Fit baseline OLS
lr = LinearRegression()

lr.fit(X_train, y_train)

# Predict
y_pred_lr = lr.predict(X_test)

# Compute RMSE manually
#rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
#print("RMSE:", rmse)

# Evaluation

print("Baseline Linear Regression")
print("R-squared:", r2_score(y_test, y_pred_lr))
# Compute RMSE manually
#print("RMSE:", mean_squared_error(y_test, y_pred_lr, squared=False))
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print("RMSE:", rmse)


In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

# Ridge Regression (L2)
ridge = Ridge()
ridge_params = {'alpha': [0.01, 0.1, 1, 10, 50, 100]}
ridge_cv = GridSearchCV(ridge, ridge_params, cv=5)
ridge_cv.fit(X_train, y_train)
y_pred_ridge = ridge_cv.predict(X_test)

print("Best Ridge alpha:", ridge_cv.best_params_)
print("Ridge R-squared:", r2_score(y_test, y_pred_ridge))
# Compute RMSE manually
#print("RMSE:", mean_squared_error(y_test, y_pred_lr, squared=False))
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print("RMSE:", rmse)


In [ ]:
# Lasso Regression (L1)
lasso = Lasso(max_iter=10000)
lasso_params = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
lasso_cv = GridSearchCV(lasso, lasso_params, cv=5)
lasso_cv.fit(X_train, y_train)
y_pred_lasso = lasso_cv.predict(X_test)

print("Best Lasso alpha:", lasso_cv.best_params_)
print("Lasso R-squared:", r2_score(y_test, y_pred_lasso))
# Compute RMSE manually
#print("RMSE:", mean_squared_error(y_test, y_pred_lr, squared=False))
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print("RMSE:", rmse)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest R-squared:", r2_score(y_test, y_pred_rf))

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Random Forest RMSE:", rmse_rf)




In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

# R2 cross-validation
cv_r2 = cross_val_score(
    rf,
    X,
    y,
    cv=5,
    scoring="r2"
)

# RMSE cross-validation (negative MSE -> RMSE)
cv_rmse = -cross_val_score(
    rf,
    X,
    y,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

print("Cross-validated R2 scores:", cv_r2)
print("Mean CV R2:", cv_r2.mean())
print("Std CV R2:", cv_r2.std())

print("\nCross-validated RMSE scores:", cv_rmse)
print("Mean CV RMSE:", cv_rmse.mean())
print("Std CV RMSE:", cv_rmse.std())


# Model comparison:

# Model	                         R²	            RMSE           CV R² (Mean)	       Interpretation

Baseline Linear Regression	     0.784	        5,796.3	       -                   Strong linear fit but large absolute                                                                                      errors due to outliers

Ridge Regression (α=1)	         0.783	        5,796.3	       -                   No practical improvement over OLS

Lasso Regression (α=10)	         0.783	        5,796.3	       -                   No meaningful regularization effect

Random Forest	                 0.864	        4,586.9	       0.837               Massively superior predictive accuracy

# Selection of the best-performing model

Random Forest is unequivocally the best-performing model for my stated goal: predicting raw charges in the presence of outliers.

# Why this conclusion is justified

Highest predictive accuracy, lowest error, stable cross-validated performance.

RMSE drops by ~21% compared to linear models (5,796 → 4,587).

R² increases from ~0.784 to 0.864, indicating near-complete variance capture.

Tree-based models are robust to non-linearity, interactions, and extreme values, which dominate insurance charge data.

Linear, Ridge, and Lasso models all:

    Produce essentially identical RMSEs

    Fail to reduce large prediction errors

    Are constrained by linearity and squared-error sensitivity to outliers

# Decision

    Primary predictive model: Random Forest

    Justification: Lowest RMSE, highest R², good to handle outliers and non-linear effects

    Models rejected for prediction: OLS, Ridge, Lasso.


# Model performance

Among the evaluated models, the Random Forest regression outperformed all linear and regularized regressions. While linear, Ridge, and Lasso models achieved R² ≈ 0.784 with RMSE ≈ 5,796, the Random Forest achieved an R² of 0.864 and an RMSE of 4,587. Given the objective of predicting raw insurance charges in the presence of extreme outliers, the Random Forest model was selected as the final predictive model.

In [ ]:
# Residuals
residuals_rf = y_test - y_pred_rf

#Residuals vs Predicted
plt.figure(figsize=(7,5))
plt.scatter(y_pred_rf, residuals_rf, alpha=0.5)
plt.axhline(0)
plt.xlabel("Predicted charges")
plt.ylabel("Residuals")
plt.title("Random Forest: Residuals vs Predicted")
plt.show()

In [ ]:
#Residual distribution
plt.figure(figsize=(7,5))
plt.hist(residuals_rf, bins=40)
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.title("Random Forest: Residual Distribution")
plt.show()

In [ ]:
feature_importance = pd.DataFrame({
    "Feature": X.columns,
    "Importance": rf.feature_importances_
}).sort_values(by="Importance", ascending=False)

feature_importance

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(feature_importance["Feature"], feature_importance["Importance"])
plt.xlabel("Importance")
plt.title("Random Forest Feature Importance")
plt.gca().invert_yaxis()
plt.show()


# Analysis of Random Forest Feature Importance

Feature importance analysis from the Random Forest model indicates that smoking status is the dominant predictor of insurance charges, accounting for approximately 61% of the total importance. 

Body mass index (21%) and age (13%) were the next most influential variables, while the number of children, sex, and region contributed minimally to predictive performance. 

This pattern suggests that insurance charges are primarily driven by health-related risk factors rather than demographic or geographic characteristics.

# Limitations of Random Forest

First, the Random Forest model does not yield interpretable coefficients, p-values, or confidence intervals, limiting causal inference. 

Second, residuals exhibit heteroskedasticity at higher predicted charges, indicating reduced accuracy for extreme-cost cases. 

Third, the dataset lacks behavioural and medical variables (e.g., chronic conditions), which may explain remaining unexplained variance. 

Finally, feature importance measures reflect predictive contribution rather than causal effect and should be interpreted accordingly.

# Final Verdict of Random Forest

Best predictive model: Random Forest

Best inferential model: Linear regression

Cross-validation confirms generalisation

No leakage, no overfitting

# General Interpretation and Insight 

Which variables matter most?

Model results consistently show that a small number of variables dominate the prediction of insurance charges:

    Smoking status
Smoking is by far the most influential variable, accounting for approximately 61% of total feature importance in the Random Forest model. This indicates that smokers face substantially higher expected insurance charges, dwarfing the effects of all other observed characteristics.

    Body Mass Index (BMI)
BMI is the second most important predictor (≈21%). Higher BMI values are associated with increased healthcare utilisation and chronic disease risk, contributing meaningfully to higher insurance charges.

    Age
Age explains approximately 13–14% of model importance. Charges increase steadily with age, reflecting rising health risks and medical expenditures over the life cycle. This aligns with the earlier linear regression results that showed a statistically significant positive age coefficient.

    Children, sex, and region
These variables contribute minimally to predictive performance. Their low importance suggests limited explanatory power once smoking, BMI, and age are accounted for.

# Key takeaway
Insurance pricing in this dataset is driven primarily by health risk behaviours and physiological risk factors, not demographic or geographic characteristics.

# What does this imply for pricing fairness and incentives?

    Pricing fairness

The dominant role of smoking, BMI, and age suggests that pricing is largely aligned with risk-based actuarial principles.

Minimal importance of sex and region indicates limited reliance on characteristics that are often considered ethically sensitive or legally restricted.

Age remains a sensitive variable: while actuarially justified, excessive age-based pricing can raise equity concerns, particularly for older individuals with limited income flexibility.

Overall, the model suggests a relatively fair pricing structure, as charges reflect modifiable health risks rather than immutable personal attributes.

    Incentive effects

The strong impact of smoking creates a clear financial incentive for smoking cessation. Premium differentials could motivate behavioural change if paired with support programs.

The substantial role of BMI implies that insurers may incentivise preventive health measures such as weight management, fitness programs, or nutritional interventions.

Because demographic variables have little influence, behavioural change appears more impactful than demographic circumstances in reducing expected charges.

# Policy Implications

Preventive health policies (smoking cessation, obesity reduction) are likely to be more effective and equitable than demographic-based pricing adjustments.

Insurers could justify wellness-linked premium discounts without introducing discriminatory pricing.

Regulators should monitor age-based pricing to ensure affordability while allowing risk-reflective premiums.

# Explicit Project Limitations

No clinical diagnoses

No utilization measures
